In [1]:
import pandas as pd
import numpy as np
import os
import requests
import zipfile
from datetime import datetime
#import geopandas as gpd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist, pdist
import statsmodels.formula.api as smf
import urllib
import matplotlib.cm as cm
from sklearn import metrics
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('master2_2010_2017.csv')
data

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,bbl,geoid,year_x,borough,lot,cd,zipcode,zonedist1,zonedist2,...,income_pc,housing_units,geometry,subway_acc,subway,subway_scale,%low_educ,noisePerCapita,tractid,built_green
0,0,2.025830e+09,360050019004,2010,BX,2,201,0,M3-1,NaN,...,0.0,0.0,POINT (-73.91419443859854 40.79864859514597),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
1,1,2.025430e+09,360050019004,2010,BX,1,201,10454,M3-1,M2-1,...,0.0,0.0,POINT (-73.91862272941451 40.80038462661332),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
2,2,2.022600e+09,360050019004,2010,BX,100,201,10454,M3-1,NaN,...,0.0,0.0,POINT (-73.92942328763192 40.80654795473313),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
3,3,2.025831e+09,360050019004,2010,BX,650,201,0,M3-1,NaN,...,0.0,0.0,POINT (-73.91292003032608 40.79693680826438),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
4,4,2.025830e+09,360050019004,2010,BX,365,201,10454,M3-1,NaN,...,0.0,0.0,POINT (-73.90968640545535 40.79845007926411),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946827,6946827,5.012010e+09,360850223003,2017,SI,1,501,0,PARK,NaN,...,0.0,0.0,POINT (-74.15950861367929 40.6430552947042),0.000000,0.169509,0.169509,NaN,0.000000,36085022300,0.286861
6946828,6946828,5.030720e+09,360850008003,2017,SI,13,501,10305,R3-2,NaN,...,0.0,565.0,POINT (-74.06586540121242 40.60548266678017),664.483398,0.180774,0.180774,0.353465,0.000886,36085000800,0.333658
6946829,6946829,5.038330e+09,360850112010,2017,SI,500,595,10305,PARK,NaN,...,0.0,0.0,POINT (-74.08363677777245 40.57257314745224),121.253517,0.171565,0.171565,NaN,0.000000,36085011201,0.473263
6946830,6946830,5.011050e+09,360850207000,2017,SI,1,501,10302,M2-1,M1-1,...,21737.0,603.0,POINT (-74.14005172774624 40.64023587350717),-9999.000000,0.000000,0.000000,0.542117,0.000000,36085020700,0.311456


In [3]:
data = data.drop_duplicates()
data

,Unnamed: 0,bbl,geoid,year_x,borough,lot,cd,zipcode,zonedist1,zonedist2,...,income_pc,housing_units,geometry,subway_acc,subway,subway_scale,%low_educ,noisePerCapita,tractid,built_green
0,0,2.025830e+09,360050019004,2010,BX,2,201,0,M3-1,NaN,...,0.0,0.0,POINT (-73.91419443859854 40.79864859514597),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
1,1,2.025430e+09,360050019004,2010,BX,1,201,10454,M3-1,M2-1,...,0.0,0.0,POINT (-73.91862272941451 40.80038462661332),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
2,2,2.022600e+09,360050019004,2010,BX,100,201,10454,M3-1,NaN,...,0.0,0.0,POINT (-73.92942328763192 40.80654795473313),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
3,3,2.025831e+09,360050019004,2010,BX,650,201,0,M3-1,NaN,...,0.0,0.0,POINT (-73.91292003032608 40.79693680826438),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
4,4,2.025830e+09,360050019004,2010,BX,365,201,10454,M3-1,NaN,...,0.0,0.0,POINT (-73.90968640545535 40.79845007926411),7720.114746,0.300386,0.300386,NaN,0.000000,36005001900,0.158909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946827,6946827,5.012010e+09,360850223003,2017,SI,1,501,0,PARK,NaN,...,0.0,0.0,POINT (-74.15950861367929 40.6430552947042),0.000000,0.169509,0.169509,NaN,0.000000,36085022300,0.286861
6946828,6946828,5.030720e+09,360850008003,2017,SI,13,501,10305,R3-2,NaN,...,0.0,565.0,POINT (-74.06586540121242 40.60548266678017),664.483398,0.180774,0.180774,0.353465,0.000886,36085000800,0.333658
6946829,6946829,5.038330e+09,360850112010,2017,SI,500,595,10305,PARK,NaN,...,0.0,0.0,POINT (-74.08363677777245 40.57257314745224),121.253517,0.171565,0.171565,NaN,0.000000,36085011201,0.473263
6946830,6946830,5.011050e+09,360850207000,2017,SI,1,501,10302,M2-1,M1-1,...,21737.0,603.0,POINT (-74.14005172774624 40.64023587350717),-9999.000000,0.000000,0.000000,0.542117,0.000000,36085020700,0.311456


In [4]:
data.year_x.unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [5]:
data1 = data[data['year_x'] == 2010]
data1 = data1[['bbl', 'geoid', 'tractid', 'geometry']]
data1

,bbl,geoid,tractid,geometry
0,2.025830e+09,360050019004,36005001900,POINT (-73.91419443859854 40.79864859514597)
1,2.025430e+09,360050019004,36005001900,POINT (-73.91862272941451 40.80038462661332)
2,2.022600e+09,360050019004,36005001900,POINT (-73.92942328763192 40.80654795473313)
3,2.025831e+09,360050019004,36005001900,POINT (-73.91292003032608 40.79693680826438)
4,2.025830e+09,360050019004,36005001900,POINT (-73.90968640545535 40.79845007926411)
...,...,...,...,...
869502,5.031110e+09,360850074001,36085007400,POINT (-74.06725041866017 40.59396355636306)
869503,5.031220e+09,360850074001,36085007400,POINT (-74.06682952527733 40.59236060995881)
869504,5.031110e+09,360850074001,36085007400,POINT (-74.0668888598582 40.59329158087536)
869505,5.031220e+09,360850074001,36085007400,POINT (-74.0661914388354 40.59252777016009)


In [6]:
data2 = data[['bbl', 'year_x', 'zipcode', 
       'comarea', 'resarea', 'retailarea', 'garagearea', 
       'unitsres', 'assessland', 'assesstot',
       'builtfar', 'residfar', 'commfar', 'facilfar', 'population',
       'income_pc', 'housing_units', 'subway_acc',
       '%low_educ', 'noisePerCapita','built_green' 
       ]]  

In [7]:
soho = data2.loc[data2['zipcode'].isin([10012,10013])]
soho

,bbl,year_x,zipcode,comarea,resarea,retailarea,garagearea,unitsres,assessland,assesstot,...,residfar,commfar,facilfar,population,income_pc,housing_units,subway_acc,%low_educ,noisePerCapita,built_green
377078,1.001840e+09,2010,10013,0,0,0,0,0,5130000.0,5136975.0,...,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377079,1.001840e+09,2010,10013,168830,0,0,0,0,8550000.0,9675000.0,...,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377080,1.001840e+09,2010,10013,98000,0,0,0,0,10575000.0,11385000.0,...,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377081,1.001840e+09,2010,10013,0,0,0,0,0,18270000.0,18270000.0,...,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377082,1.002160e+09,2010,10013,760000,0,0,0,0,8460000.0,43380000.0,...,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492729,1.001640e+09,2017,10013,6100,4600,3800,0,2,202500.0,652950.0,...,3.44,6.0,6.5,1465.0,118657.0,408.0,46517.480469,0.500341,0.000143,0.142875
6493757,1.005330e+09,2017,10012,0,0,0,0,0,3298500.0,3298500.0,...,6.02,2.0,6.5,1795.0,0.0,1233.0,46853.386719,0.056825,0.000000,0.269999
6493758,1.005330e+09,2017,10012,32700,0,16350,0,0,1539000.0,6299100.0,...,6.02,2.0,6.5,1795.0,0.0,1233.0,46853.386719,0.056825,0.000000,0.269999
6493759,1.005330e+09,2017,10012,140481,0,0,121624,1904,8190450.0,38920950.0,...,6.02,2.0,6.5,1795.0,0.0,1233.0,46853.386719,0.056825,0.000000,0.269999


In [8]:
gowanus = data2.loc[data2['zipcode'].isin([11215, 11217, 11231])]
gowanus

,bbl,year_x,zipcode,comarea,resarea,retailarea,garagearea,unitsres,assessland,assesstot,...,residfar,commfar,facilfar,population,income_pc,housing_units,subway_acc,%low_educ,noisePerCapita,built_green
89964,3.005140e+09,2010,11231,3294,0,0,0,0,522000.0,580500.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89965,3.005140e+09,2010,11231,3294,0,0,0,0,522000.0,580500.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89966,3.005850e+09,2010,11231,0,5500,0,0,6,136.0,13565.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89967,3.005850e+09,2010,11231,0,5500,0,0,6,136.0,13565.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89968,3.005850e+09,2010,11231,0,1875,0,0,1,4582.0,8210.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6450906,3.003920e+09,2017,11217,0,448000,0,0,392,2291400.0,7335450.0,...,2.43,0.0,4.8,1139.0,9541.0,591.0,26879.044922,0.755926,0.044545,0.308133
6450907,3.003920e+09,2017,11217,400,0,0,0,0,889650.0,1014300.0,...,0.00,0.0,0.0,1139.0,9541.0,591.0,26879.044922,0.755926,0.044545,0.308133
6450921,3.006230e+09,2017,11231,42222,0,0,0,0,372600.0,924750.0,...,0.00,2.0,0.0,296.0,0.0,195.0,2615.246582,0.290541,0.000000,0.144824
6450922,3.006240e+09,2017,11217,0,0,0,0,0,3600.0,3600.0,...,0.00,2.0,0.0,296.0,0.0,195.0,2615.246582,0.290541,0.000000,0.144824


In [9]:
soho = soho.drop_duplicates()
gowanus = gowanus.drop_duplicates()
gowanus

,bbl,year_x,zipcode,comarea,resarea,retailarea,garagearea,unitsres,assessland,assesstot,...,residfar,commfar,facilfar,population,income_pc,housing_units,subway_acc,%low_educ,noisePerCapita,built_green
89964,3.005140e+09,2010,11231,3294,0,0,0,0,522000.0,580500.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89966,3.005850e+09,2010,11231,0,5500,0,0,6,136.0,13565.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89968,3.005850e+09,2010,11231,0,1875,0,0,1,4582.0,8210.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89970,3.005730e+09,2010,11231,86916,0,0,0,0,264150.0,1863000.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89972,3.005140e+09,2010,11231,0,0,0,0,0,411840.0,414446.0,...,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6450906,3.003920e+09,2017,11217,0,448000,0,0,392,2291400.0,7335450.0,...,2.43,0.0,4.8,1139.0,9541.0,591.0,26879.044922,0.755926,0.044545,0.308133
6450907,3.003920e+09,2017,11217,400,0,0,0,0,889650.0,1014300.0,...,0.00,0.0,0.0,1139.0,9541.0,591.0,26879.044922,0.755926,0.044545,0.308133
6450921,3.006230e+09,2017,11231,42222,0,0,0,0,372600.0,924750.0,...,0.00,2.0,0.0,296.0,0.0,195.0,2615.246582,0.290541,0.000000,0.144824
6450922,3.006240e+09,2017,11217,0,0,0,0,0,3600.0,3600.0,...,0.00,2.0,0.0,296.0,0.0,195.0,2615.246582,0.290541,0.000000,0.144824


In [10]:
soho2010 = soho[soho['year_x'] == 2010].drop(['year_x','zipcode'], axis=1)
soho2011 = soho[soho['year_x'] == 2011].drop(['year_x','zipcode'], axis=1)
soho2012 = soho[soho['year_x'] == 2012].drop(['year_x','zipcode'], axis=1)
soho2013 = soho[soho['year_x'] == 2013].drop(['year_x','zipcode'], axis=1)
soho2014 = soho[soho['year_x'] == 2014].drop(['year_x','zipcode'], axis=1)
soho2015 = soho[soho['year_x'] == 2015].drop(['year_x','zipcode'], axis=1)
soho2016 = soho[soho['year_x'] == 2016].drop(['year_x','zipcode'], axis=1)
soho2017 = soho[soho['year_x'] == 2017].drop(['year_x','zipcode'], axis=1)
soho2010

,bbl,comarea,resarea,retailarea,garagearea,unitsres,assessland,assesstot,builtfar,residfar,commfar,facilfar,population,income_pc,housing_units,subway_acc,%low_educ,noisePerCapita,built_green
377078,1.001840e+09,0,0,0,0,0,5130000.0,5136975.0,0.00,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377079,1.001840e+09,168830,0,0,0,0,8550000.0,9675000.0,1.19,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377080,1.001840e+09,98000,0,0,0,0,10575000.0,11385000.0,0.66,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377081,1.001840e+09,0,0,0,0,0,18270000.0,18270000.0,0.00,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
377082,1.002160e+09,760000,0,0,0,0,8460000.0,43380000.0,8.44,NaN,NaN,NaN,1207.0,35340.0,949.0,35580.480469,0.218724,0.000368,0.114993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389986,1.005310e+09,1758,3517,1758,0,2,52806.0,87920.0,2.62,NaN,NaN,NaN,477.0,91081.0,385.0,43359.796875,0.071279,0.000220,0.086855
389987,1.005310e+09,733,0,0,733,0,315450.0,356400.0,0.47,NaN,NaN,NaN,477.0,91081.0,385.0,43359.796875,0.071279,0.000220,0.086855
389988,1.005300e+09,1619,4857,1619,0,3,119445.0,206980.0,2.42,NaN,NaN,NaN,477.0,91081.0,385.0,43359.796875,0.071279,0.000220,0.086855
389989,1.005300e+09,2400,11685,2400,0,7,181488.0,482680.0,5.42,NaN,NaN,NaN,477.0,91081.0,385.0,43359.796875,0.071279,0.000220,0.086855


In [11]:
soho2010 = soho2010.rename(columns={
    'comarea' : 'comarea10', 
    'resarea' : 'resarea10', 
    'retailarea' : 'retailarea10', 
    'garagearea' : 'garagearea10', 
    'unitsres' : 'unitsres10', 
    'assessland' : 'assessland10', 
    'assesstot' : 'assesstot10',
    'builtfar' : 'builtfar10', 
    'residfar' : 'residfar10', 
    'commfar' : 'commfar10', 
    'facilfar' : 'facilfar10', 
    'population' : 'population10',
    'income_pc' : 'income_pc10', 
    'housing_units' : 'housing_units10', 
    'subway_acc' : 'subway_acc10',
    '%low_educ' : '%low_educ10', 
    'noisePerCapita' : 'noisePerCapita10',
    'built_green' : 'built_green10'})
soho2011 = soho2011.rename(columns={
    'comarea' : 'comarea11', 
    'resarea' : 'resarea11', 
    'retailarea' : 'retailarea11', 
    'garagearea' : 'garagearea11', 
    'unitsres' : 'unitsres11', 
    'assessland' : 'assessland11', 
    'assesstot' : 'assesstot11',
    'builtfar' : 'builtfar11', 
    'residfar' : 'residfar11', 
    'commfar' : 'commfar11', 
    'facilfar' : 'facilfar11', 
    'population' : 'population11',
    'income_pc' : 'income_pc11', 
    'housing_units' : 'housing_units11', 
    'subway_acc' : 'subway_acc11',
    '%low_educ' : '%low_educ11', 
    'noisePerCapita' : 'noisePerCapita11',
    'built_green' : 'built_green11'})
soho2012 = soho2012.rename(columns={
    'comarea' : 'comarea12', 
    'resarea' : 'resarea12', 
    'retailarea' : 'retailarea12', 
    'garagearea' : 'garagearea12', 
    'unitsres' : 'unitsres12', 
    'assessland' : 'assessland12', 
    'assesstot' : 'assesstot12',
    'builtfar' : 'builtfar12', 
    'residfar' : 'residfar12', 
    'commfar' : 'commfar12', 
    'facilfar' : 'facilfar12', 
    'population' : 'population12',
    'income_pc' : 'income_pc12', 
    'housing_units' : 'housing_units12', 
    'subway_acc' : 'subway_acc12',
    '%low_educ' : '%low_educ12', 
    'noisePerCapita' : 'noisePerCapita12',
    'built_green' : 'built_green12'})
soho2013 = soho2013.rename(columns={
    'comarea' : 'comarea13', 
    'resarea' : 'resarea13', 
    'retailarea' : 'retailarea13', 
    'garagearea' : 'garagearea13', 
    'unitsres' : 'unitsres13', 
    'assessland' : 'assessland13', 
    'assesstot' : 'assesstot13',
    'builtfar' : 'builtfar13', 
    'residfar' : 'residfar13', 
    'commfar' : 'commfar13', 
    'facilfar' : 'facilfar13', 
    'population' : 'population13',
    'income_pc' : 'income_pc13', 
    'housing_units' : 'housing_units13', 
    'subway_acc' : 'subway_acc13',
    '%low_educ' : '%low_educ13', 
    'noisePerCapita' : 'noisePerCapita13',
    'built_green' : 'built_green13'})
soho2014 = soho2014.rename(columns={
    'comarea' : 'comarea14', 
    'resarea' : 'resarea14', 
    'retailarea' : 'retailarea14', 
    'garagearea' : 'garagearea14', 
    'unitsres' : 'unitsres14', 
    'assessland' : 'assessland14', 
    'assesstot' : 'assesstot14',
    'builtfar' : 'builtfar14', 
    'residfar' : 'residfar14', 
    'commfar' : 'commfar14', 
    'facilfar' : 'facilfar14', 
    'population' : 'population14',
    'income_pc' : 'income_pc14', 
    'housing_units' : 'housing_units14', 
    'subway_acc' : 'subway_acc14',
    '%low_educ' : '%low_educ14', 
    'noisePerCapita' : 'noisePerCapita14',
    'built_green' : 'built_green14'})
soho2015 = soho2015.rename(columns={
    'comarea' : 'comarea15', 
    'resarea' : 'resarea15', 
    'retailarea' : 'retailarea15', 
    'garagearea' : 'garagearea15', 
    'unitsres' : 'unitsres15', 
    'assessland' : 'assessland15', 
    'assesstot' : 'assesstot15',
    'builtfar' : 'builtfar15', 
    'residfar' : 'residfar15', 
    'commfar' : 'commfar15', 
    'facilfar' : 'facilfar15', 
    'population' : 'population15',
    'income_pc' : 'income_pc15', 
    'housing_units' : 'housing_units15', 
    'subway_acc' : 'subway_acc15',
    '%low_educ' : '%low_educ15', 
    'noisePerCapita' : 'noisePerCapita15',
    'built_green' : 'built_green15'})
soho2016 = soho2016.rename(columns={
    'comarea' : 'comarea16', 
    'resarea' : 'resarea16', 
    'retailarea' : 'retailarea16', 
    'garagearea' : 'garagearea16', 
    'unitsres' : 'unitsres16', 
    'assessland' : 'assessland16', 
    'assesstot' : 'assesstot16',
    'builtfar' : 'builtfar16', 
    'residfar' : 'residfar16', 
    'commfar' : 'commfar16', 
    'facilfar' : 'facilfar16', 
    'population' : 'population16',
    'income_pc' : 'income_pc16', 
    'housing_units' : 'housing_units16', 
    'subway_acc' : 'subway_acc16',
    '%low_educ' : '%low_educ16', 
    'noisePerCapita' : 'noisePerCapita16',
    'built_green' : 'built_green16'})
soho2017 = soho2017.rename(columns={
    'comarea' : 'comarea17', 
    'resarea' : 'resarea17', 
    'retailarea' : 'retailarea17', 
    'garagearea' : 'garagearea17', 
    'unitsres' : 'unitsres17', 
    'assessland' : 'assessland17', 
    'assesstot' : 'assesstot17',
    'builtfar' : 'builtfar17', 
    'residfar' : 'residfar17', 
    'commfar' : 'commfar17', 
    'facilfar' : 'facilfar17', 
    'population' : 'population17',
    'income_pc' : 'income_pc17', 
    'housing_units' : 'housing_units17', 
    'subway_acc' : 'subway_acc17',
    '%low_educ' : '%low_educ17', 
    'noisePerCapita' : 'noisePerCapita17',
    'built_green' : 'built_green17'})
soho2017

,bbl,comarea17,resarea17,retailarea17,garagearea17,unitsres17,assessland17,assesstot17,builtfar17,residfar17,commfar17,facilfar17,population17,income_pc17,housing_units17,subway_acc17,%low_educ17,noisePerCapita17,built_green17
6451239,1.002360e+09,900,1833,900,0,2,11372.0,29859.0,1.92,6.02,6.0,6.5,978.0,43774.0,791.0,46566.289062,0.358896,0.001896,0.054816
6451240,1.002060e+09,2000,6422,2000,0,18,135000.0,989550.0,3.34,6.02,6.0,6.5,978.0,43774.0,791.0,46566.289062,0.358896,0.001896,0.054816
6451241,1.004710e+09,3047,1785,1060,0,2,121500.0,621450.0,4.53,6.02,6.0,6.5,978.0,43774.0,791.0,46566.289062,0.358896,0.001896,0.054816
6451242,1.002350e+09,1050,4200,1050,0,5,113154.0,386751.0,3.57,0.00,5.0,6.5,978.0,43774.0,791.0,46566.289062,0.358896,0.001896,0.054816
6451243,1.002360e+09,0,9875,0,0,20,171000.0,445050.0,3.92,6.02,6.0,6.5,978.0,43774.0,791.0,46566.289062,0.358896,0.001896,0.054816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492729,1.001640e+09,6100,4600,3800,0,2,202500.0,652950.0,4.65,3.44,6.0,6.5,1465.0,118657.0,408.0,46517.480469,0.500341,0.000143,0.142875
6493757,1.005330e+09,0,0,0,0,0,3298500.0,3298500.0,0.00,6.02,2.0,6.5,1795.0,0.0,1233.0,46853.386719,0.056825,0.000000,0.269999
6493758,1.005330e+09,32700,0,16350,0,0,1539000.0,6299100.0,1.47,6.02,2.0,6.5,1795.0,0.0,1233.0,46853.386719,0.056825,0.000000,0.269999
6493759,1.005330e+09,140481,0,0,121624,1904,8190450.0,38920950.0,5.60,6.02,2.0,6.5,1795.0,0.0,1233.0,46853.386719,0.056825,0.000000,0.269999


In [12]:
data3=pd.merge(data1, soho2010, on='bbl')
data4=pd.merge(data3, soho2011, on='bbl')
data5=pd.merge(data4, soho2012, on='bbl')
data6=pd.merge(data5, soho2013, on='bbl')
data7=pd.merge(data6, soho2014, on='bbl')
data8=pd.merge(data7, soho2015, on='bbl')
data9=pd.merge(data8, soho2016, on='bbl')
data10=pd.merge(data9, soho2017, on='bbl')
data10

,bbl,geoid,tractid,geometry,comarea10,resarea10,retailarea10,garagearea10,unitsres10,assessland10,...,residfar17,commfar17,facilfar17,population17,income_pc17,housing_units17,subway_acc17,%low_educ17,noisePerCapita17,built_green17
0,1.001840e+09,360610039002,36061003900,POINT (-74.01298051461913 40.71932460853346),0,0,0,0,0,5130000.0,...,0.0,2.0,0.0,999.0,0.0,811.0,35580.480469,0.215215,0.0,0.114993
1,1.001840e+09,360610039002,36061003900,POINT (-74.01350256967636 40.72019166528532),168830,0,0,0,0,8550000.0,...,0.0,2.0,0.0,999.0,0.0,811.0,35580.480469,0.215215,0.0,0.114993
2,1.001840e+09,360610039002,36061003900,POINT (-74.01286224946163 40.72113078270549),98000,0,0,0,0,10575000.0,...,0.0,2.0,0.0,999.0,0.0,811.0,35580.480469,0.215215,0.0,0.114993
3,1.001840e+09,360610039002,36061003900,POINT (-74.01240136757583 40.72218752379005),0,0,0,0,0,18270000.0,...,0.0,2.0,0.0,999.0,0.0,811.0,35580.480469,0.215215,0.0,0.114993
4,1.002160e+09,360610039002,36061003900,POINT (-74.01096149562558 40.72114194358648),760000,0,0,0,0,8460000.0,...,10.0,10.0,10.0,999.0,0.0,811.0,35580.480469,0.215215,0.0,0.114993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1.005310e+09,360610055021,36061005502,POINT (-73.99200806815747 40.72681028708914),1758,3517,1758,0,2,52806.0,...,0.0,5.0,6.5,556.0,0.0,364.0,43359.796875,0.064748,0.0,0.086855
2667,1.005310e+09,360610055021,36061005502,POINT (-73.9920239893991 40.72664274470679),733,0,0,733,0,315450.0,...,0.0,5.0,6.5,556.0,0.0,364.0,43359.796875,0.064748,0.0,0.086855
2668,1.005300e+09,360610055021,36061005502,POINT (-73.99347926979189 40.7265884803901),1619,4857,1619,0,3,119445.0,...,0.0,5.0,6.5,556.0,0.0,364.0,43359.796875,0.064748,0.0,0.086855
2669,1.005300e+09,360610055021,36061005502,POINT (-73.99278626657211 40.72659141116476),2400,11685,2400,0,7,181488.0,...,0.0,5.0,6.5,556.0,0.0,364.0,43359.796875,0.064748,0.0,0.086855


In [13]:
data10.to_csv('soho1017.csv')

In [14]:
gowanus2010 = gowanus[gowanus['year_x'] == 2010].drop(['year_x','zipcode'], axis=1)
gowanus2011 = gowanus[gowanus['year_x'] == 2011].drop(['year_x','zipcode'], axis=1)
gowanus2012 = gowanus[gowanus['year_x'] == 2012].drop(['year_x','zipcode'], axis=1)
gowanus2013 = gowanus[gowanus['year_x'] == 2013].drop(['year_x','zipcode'], axis=1)
gowanus2014 = gowanus[gowanus['year_x'] == 2014].drop(['year_x','zipcode'], axis=1)
gowanus2015 = gowanus[gowanus['year_x'] == 2015].drop(['year_x','zipcode'], axis=1)
gowanus2016 = gowanus[gowanus['year_x'] == 2016].drop(['year_x','zipcode'], axis=1)
gowanus2017 = gowanus[gowanus['year_x'] == 2017].drop(['year_x','zipcode'], axis=1)
gowanus2010

,bbl,comarea,resarea,retailarea,garagearea,unitsres,assessland,assesstot,builtfar,residfar,commfar,facilfar,population,income_pc,housing_units,subway_acc,%low_educ,noisePerCapita,built_green
89964,3.005140e+09,3294,0,0,0,0,522000.0,580500.0,0.07,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89966,3.005850e+09,0,5500,0,0,6,136.0,13565.0,2.20,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89968,3.005850e+09,0,1875,0,0,1,4582.0,8210.0,0.75,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89970,3.005730e+09,86916,0,0,0,0,264150.0,1863000.0,1.02,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
89972,3.005140e+09,0,0,0,0,0,411840.0,414446.0,0.00,NaN,NaN,NaN,230.0,57733.0,150.0,2031.234619,0.073913,0.000225,0.144824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287835,3.020990e+09,0,3360,0,0,3,28097.0,51023.0,1.93,NaN,NaN,NaN,885.0,58353.0,517.0,26069.138672,0.084746,0.000120,0.164739
287836,3.020990e+09,700,2100,0,0,3,21284.0,31111.0,1.75,NaN,NaN,NaN,885.0,58353.0,517.0,26069.138672,0.084746,0.000120,0.164739
287837,3.020990e+09,0,3600,0,0,4,3997.0,108218.0,1.80,NaN,NaN,NaN,885.0,58353.0,517.0,26069.138672,0.084746,0.000120,0.164739
287838,3.020990e+09,1600,2360,1600,0,2,22894.0,41520.0,2.48,NaN,NaN,NaN,885.0,58353.0,517.0,26069.138672,0.084746,0.000120,0.164739


In [15]:
gowanus2010 = gowanus2010.rename(columns={
    'comarea' : 'comarea10', 
    'resarea' : 'resarea10', 
    'retailarea' : 'retailarea10', 
    'garagearea' : 'garagearea10', 
    'unitsres' : 'unitsres10', 
    'assessland' : 'assessland10', 
    'assesstot' : 'assesstot10',
    'builtfar' : 'builtfar10', 
    'residfar' : 'residfar10', 
    'commfar' : 'commfar10', 
    'facilfar' : 'facilfar10', 
    'population' : 'population10',
    'income_pc' : 'income_pc10', 
    'housing_units' : 'housing_units10', 
    'subway_acc' : 'subway_acc10',
    '%low_educ' : '%low_educ10', 
    'noisePerCapita' : 'noisePerCapita10',
    'built_green' : 'built_green10'})
gowanus2011 = gowanus2011.rename(columns={
    'comarea' : 'comarea11', 
    'resarea' : 'resarea11', 
    'retailarea' : 'retailarea11', 
    'garagearea' : 'garagearea11', 
    'unitsres' : 'unitsres11', 
    'assessland' : 'assessland11', 
    'assesstot' : 'assesstot11',
    'builtfar' : 'builtfar11', 
    'residfar' : 'residfar11', 
    'commfar' : 'commfar11', 
    'facilfar' : 'facilfar11', 
    'population' : 'population11',
    'income_pc' : 'income_pc11', 
    'housing_units' : 'housing_units11', 
    'subway_acc' : 'subway_acc11',
    '%low_educ' : '%low_educ11', 
    'noisePerCapita' : 'noisePerCapita11',
    'built_green' : 'built_green11'})
gowanus2012 = gowanus2012.rename(columns={
    'comarea' : 'comarea12', 
    'resarea' : 'resarea12', 
    'retailarea' : 'retailarea12', 
    'garagearea' : 'garagearea12', 
    'unitsres' : 'unitsres12', 
    'assessland' : 'assessland12', 
    'assesstot' : 'assesstot12',
    'builtfar' : 'builtfar12', 
    'residfar' : 'residfar12', 
    'commfar' : 'commfar12', 
    'facilfar' : 'facilfar12', 
    'population' : 'population12',
    'income_pc' : 'income_pc12', 
    'housing_units' : 'housing_units12', 
    'subway_acc' : 'subway_acc12',
    '%low_educ' : '%low_educ12', 
    'noisePerCapita' : 'noisePerCapita12',
    'built_green' : 'built_green12'})
gowanus2013 = gowanus2013.rename(columns={
    'comarea' : 'comarea13', 
    'resarea' : 'resarea13', 
    'retailarea' : 'retailarea13', 
    'garagearea' : 'garagearea13', 
    'unitsres' : 'unitsres13', 
    'assessland' : 'assessland13', 
    'assesstot' : 'assesstot13',
    'builtfar' : 'builtfar13', 
    'residfar' : 'residfar13', 
    'commfar' : 'commfar13', 
    'facilfar' : 'facilfar13', 
    'population' : 'population13',
    'income_pc' : 'income_pc13', 
    'housing_units' : 'housing_units13', 
    'subway_acc' : 'subway_acc13',
    '%low_educ' : '%low_educ13', 
    'noisePerCapita' : 'noisePerCapita13',
    'built_green' : 'built_green13'})
gowanus2014 = gowanus2014.rename(columns={
    'comarea' : 'comarea14', 
    'resarea' : 'resarea14', 
    'retailarea' : 'retailarea14', 
    'garagearea' : 'garagearea14', 
    'unitsres' : 'unitsres14', 
    'assessland' : 'assessland14', 
    'assesstot' : 'assesstot14',
    'builtfar' : 'builtfar14', 
    'residfar' : 'residfar14', 
    'commfar' : 'commfar14', 
    'facilfar' : 'facilfar14', 
    'population' : 'population14',
    'income_pc' : 'income_pc14', 
    'housing_units' : 'housing_units14', 
    'subway_acc' : 'subway_acc14',
    '%low_educ' : '%low_educ14', 
    'noisePerCapita' : 'noisePerCapita14',
    'built_green' : 'built_green14'})
gowanus2015 = gowanus2015.rename(columns={
    'comarea' : 'comarea15', 
    'resarea' : 'resarea15', 
    'retailarea' : 'retailarea15', 
    'garagearea' : 'garagearea15', 
    'unitsres' : 'unitsres15', 
    'assessland' : 'assessland15', 
    'assesstot' : 'assesstot15',
    'builtfar' : 'builtfar15', 
    'residfar' : 'residfar15', 
    'commfar' : 'commfar15', 
    'facilfar' : 'facilfar15', 
    'population' : 'population15',
    'income_pc' : 'income_pc15', 
    'housing_units' : 'housing_units15', 
    'subway_acc' : 'subway_acc15',
    '%low_educ' : '%low_educ15', 
    'noisePerCapita' : 'noisePerCapita15',
    'built_green' : 'built_green15'})
gowanus2016 = gowanus2016.rename(columns={
    'comarea' : 'comarea16', 
    'resarea' : 'resarea16', 
    'retailarea' : 'retailarea16', 
    'garagearea' : 'garagearea16', 
    'unitsres' : 'unitsres16', 
    'assessland' : 'assessland16', 
    'assesstot' : 'assesstot16',
    'builtfar' : 'builtfar16', 
    'residfar' : 'residfar16', 
    'commfar' : 'commfar16', 
    'facilfar' : 'facilfar16', 
    'population' : 'population16',
    'income_pc' : 'income_pc16', 
    'housing_units' : 'housing_units16', 
    'subway_acc' : 'subway_acc16',
    '%low_educ' : '%low_educ16', 
    'noisePerCapita' : 'noisePerCapita16',
    'built_green' : 'built_green16'})
gowanus2017 = gowanus2017.rename(columns={
    'comarea' : 'comarea17', 
    'resarea' : 'resarea17', 
    'retailarea' : 'retailarea17', 
    'garagearea' : 'garagearea17', 
    'unitsres' : 'unitsres17', 
    'assessland' : 'assessland17', 
    'assesstot' : 'assesstot17',
    'builtfar' : 'builtfar17', 
    'residfar' : 'residfar17', 
    'commfar' : 'commfar17', 
    'facilfar' : 'facilfar17', 
    'population' : 'population17',
    'income_pc' : 'income_pc17', 
    'housing_units' : 'housing_units17', 
    'subway_acc' : 'subway_acc17',
    '%low_educ' : '%low_educ17', 
    'noisePerCapita' : 'noisePerCapita17',
    'built_green' : 'built_green17'})
gowanus2017

,bbl,comarea17,resarea17,retailarea17,garagearea17,unitsres17,assessland17,assesstot17,builtfar17,residfar17,commfar17,facilfar17,population17,income_pc17,housing_units17,subway_acc17,%low_educ17,noisePerCapita17,built_green17
6170499,3.004220e+09,855,1710,855,0,2,10293.0,35690.0,2.25,2.00,0.0,2.0,628.0,0.0,425.0,21826.207031,0.111465,0.000000,0.205751
6170500,3.004220e+09,0,1824,0,0,3,5067.0,28663.0,1.54,2.00,0.0,2.0,628.0,0.0,425.0,21826.207031,0.111465,0.000000,0.205751
6170501,3.004210e+09,0,3265,0,0,4,35031.0,50629.0,1.31,2.00,0.0,2.0,628.0,0.0,425.0,21826.207031,0.111465,0.000000,0.205751
6170502,3.004220e+09,0,2400,0,0,3,5666.0,29523.0,1.60,2.00,0.0,2.0,628.0,0.0,425.0,21826.207031,0.111465,0.000000,0.205751
6170503,3.004220e+09,0,1920,0,0,3,13403.0,61119.0,1.60,2.00,0.0,2.0,628.0,0.0,425.0,21826.207031,0.111465,0.000000,0.205751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6450906,3.003920e+09,0,448000,0,0,392,2291400.0,7335450.0,1.96,2.43,0.0,4.8,1139.0,9541.0,591.0,26879.044922,0.755926,0.044545,0.308133
6450907,3.003920e+09,400,0,0,0,0,889650.0,1014300.0,0.01,0.00,0.0,0.0,1139.0,9541.0,591.0,26879.044922,0.755926,0.044545,0.308133
6450921,3.006230e+09,42222,0,0,0,0,372600.0,924750.0,0.16,0.00,2.0,0.0,296.0,0.0,195.0,2615.246582,0.290541,0.000000,0.144824
6450922,3.006240e+09,0,0,0,0,0,3600.0,3600.0,0.00,0.00,2.0,0.0,296.0,0.0,195.0,2615.246582,0.290541,0.000000,0.144824


In [16]:
data3=pd.merge(data1, gowanus2010, on='bbl')
data4=pd.merge(data3, gowanus2011, on='bbl')
data5=pd.merge(data4, gowanus2012, on='bbl')
data6=pd.merge(data5, gowanus2013, on='bbl')
data7=pd.merge(data6, gowanus2014, on='bbl')
data8=pd.merge(data7, gowanus2015, on='bbl')
data9=pd.merge(data8, gowanus2016, on='bbl')
data10=pd.merge(data9, gowanus2017, on='bbl')
data10

,bbl,geoid,tractid,geometry,comarea10,resarea10,retailarea10,garagearea10,unitsres10,assessland10,...,residfar17,commfar17,facilfar17,population17,income_pc17,housing_units17,subway_acc17,%low_educ17,noisePerCapita17,built_green17
0,3.005140e+09,360470053001,36047005300,POINT (-74.01701947529091 40.68059217001012),3294,0,0,0,0,522000.0,...,0.0,2.0,0.0,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
1,3.005140e+09,360470053001,36047005300,POINT (-74.01701947529091 40.68059217001012),3294,0,0,0,0,522000.0,...,0.0,2.0,0.0,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
2,3.005850e+09,360470053001,36047005300,POINT (-74.01559960054527 40.67752277249812),0,5500,0,0,6,136.0,...,0.0,1.0,2.4,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
3,3.005850e+09,360470053001,36047005300,POINT (-74.01559960054527 40.67752277249812),0,5500,0,0,6,136.0,...,0.0,1.0,2.4,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
4,3.005850e+09,360470053001,36047005300,POINT (-74.01620217269328 40.67793184116763),0,1875,0,0,1,4582.0,...,0.0,1.0,2.4,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18035,3.020990e+09,360470033001,36047003300,POINT (-73.97532909621347 40.6873718550666),700,1400,700,0,2,11342.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739
18036,3.020990e+09,360470033001,36047003300,POINT (-73.97574071199207 40.68933059205745),0,3360,0,0,3,28097.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739
18037,3.020990e+09,360470033001,36047003300,POINT (-73.97503624523632 40.68740423438948),700,2100,0,0,3,21284.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739
18038,3.020990e+09,360470033001,36047003300,POINT (-73.97499201273388 40.6875559867058),0,3600,0,0,4,3997.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739


In [17]:
data10 = data10.rename(columns={
    'bbl' : 'BBL', 
    'tractid' : 'Tract2000'
})
data10

,BBL,geoid,Tract2000,geometry,comarea10,resarea10,retailarea10,garagearea10,unitsres10,assessland10,...,residfar17,commfar17,facilfar17,population17,income_pc17,housing_units17,subway_acc17,%low_educ17,noisePerCapita17,built_green17
0,3.005140e+09,360470053001,36047005300,POINT (-74.01701947529091 40.68059217001012),3294,0,0,0,0,522000.0,...,0.0,2.0,0.0,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
1,3.005140e+09,360470053001,36047005300,POINT (-74.01701947529091 40.68059217001012),3294,0,0,0,0,522000.0,...,0.0,2.0,0.0,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
2,3.005850e+09,360470053001,36047005300,POINT (-74.01559960054527 40.67752277249812),0,5500,0,0,6,136.0,...,0.0,1.0,2.4,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
3,3.005850e+09,360470053001,36047005300,POINT (-74.01559960054527 40.67752277249812),0,5500,0,0,6,136.0,...,0.0,1.0,2.4,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
4,3.005850e+09,360470053001,36047005300,POINT (-74.01620217269328 40.67793184116763),0,1875,0,0,1,4582.0,...,0.0,1.0,2.4,296.0,0.0,195.0,2031.234619,0.290541,0.0,0.144824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18035,3.020990e+09,360470033001,36047003300,POINT (-73.97532909621347 40.6873718550666),700,1400,700,0,2,11342.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739
18036,3.020990e+09,360470033001,36047003300,POINT (-73.97574071199207 40.68933059205745),0,3360,0,0,3,28097.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739
18037,3.020990e+09,360470033001,36047003300,POINT (-73.97503624523632 40.68740423438948),700,2100,0,0,3,21284.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739
18038,3.020990e+09,360470033001,36047003300,POINT (-73.97499201273388 40.6875559867058),0,3600,0,0,4,3997.0,...,2.0,0.0,2.0,754.0,0.0,443.0,26069.138672,0.080902,0.0,0.164739


In [18]:
data10.to_csv('gowanus1017.csv')